Q. MNIST 데이터 셋에 대해서 Pytorch를 적용하여 모델 구성 변경, 조기 학습 중단을 수행하고 Best model 을 저장한 후 다시 불러와서 테스트 데이터로 평가한 결과를 출력하세요

In [ ]:
# torch.max(outputs, 1)

import torch

# 예시 출력 텐서
outputs = torch.tensor([[0.1, 0.3, 0.6], [0.2, 0.7, 0.1]])

# 가장 큰 값을 가지는 클래스 인덱스를 추출
_, predicted = torch.max(outputs, 1) # 각 행에 대해 최대값

print(predicted)  # tensor([2, 1])

tensor([2, 1])


In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision # torchvision에서 데이터 가져올 예정
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# MNIST 데이터셋을 위한 전처리 과정 정의
transform=transforms.Compose([
    transforms.ToTensor(), # 이미지를 PyTorch 텐서로 변환
    transforms.Normalize((0.5, ), (0.5, )) # 이미지를 평균 0.5, 표준편차 0.5로 정규화하여 [-1, 1] 범위로 조정
])

# MNIST 데이터셋 로드
train_dataset=torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset=torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    # train=True : 훈련용 데이터를 가리킴. train=False: 테스트용 데이터를 로드
    # transform=transform: 윗단에서 정의한 전처리 과정을 적용 (이미지 텐서로 변환 및 표준화)
    # 데이터셋이 로컬에 없다면 자동으로 다운로드


# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size=int(0.8*len(train_dataset)) # 훈련 세트 크기를 전체의 80%로 설정
val_size=len(train_dataset)-train_size # 검증 세트 크기 계산
train_dataset, val_dataset=random_split(train_dataset, [train_size, val_size]) # 분할 실행

trainloader=DataLoader(train_dataset, batch_size=64, shuffle=True) # shuffle=True : 훈련 데이터셋을 섞어서 모델이 특정 순서에 의존하지 않도록 함
valloader=DataLoader(val_dataset, batch_size=64, shuffle=False)
testloader=DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 아키텍처 정의
class MyModel(nn.Module): # nn.Module을 상속받아 모델 클래스를 정의
    def __init__(self): # 모델의 각 레이어를 초기화
        super().__init__()
        self.conv1= nn.Conv2d(1,20,5) # 2D 컨볼루션 레이어 정의 (입력 채널 1, 출력 채널 20, 커널 크기 5)
        self.pool= nn.MaxPool2d(2,2) # 맥스풀링 레이어 정의(2x2 풀링) - 공간 차원을 절반으로 줄임
        self.flatten = nn.Flatten() # 텐서 평탄화 (2D 이미지 텐서를 1D 벡터로 평탄화함)
        self.fc1 = nn.Linear(2880, 50) # 완전 연결 레이어 (입력 크기 2880, 출력 크기 50)
        self.fc2 = nn.Linear(50, 10) # 출력 레이어 (클래스 수 10)

    def forward(self, x): # 순전파를 정의하는 메서드
        x = self.pool(F.relu(self.conv1(x))) # ReLU 활성화 함수 적용 후 맥스풀링
        x = self.flatten(x) # 평탄화
        x = F.relu(self.fc1(x)) # ReLU 활성화 함수 적용
        x = self.fc2(x) # 최종 출력
        return x

model= MyModel() # 모델 인스턴스 생성

# 손실 함수 및 최적화 알고리즘 지정
criterion = nn.CrossEntropyLoss() # 크로스 엔트로피 손실 함수 - 모델 출력과 실제 라벨 간의 차이를 계산함
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # SGD 최적화 알고리즘

# 모델 훈련
best_val_loss = float('inf') # 검증 손실을 추적하기 위한 변수 초기화
patience, trials = 5, 0 #  조기 종료 기준 설정
num_epochs = 20 # 에폭 수 설정
for epoch in range(num_epochs):
    model.train() # 모델을 훈련 모드로 설정
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad() # 이전 배치의 그래디언트를 초기화
        outputs = model(inputs) # 모델을 통한 순전파를 수행하여 예측 값을 얻기
        loss = criterion(outputs, labels) # 손실 계산
        loss.backward() # 역전파. 손실에 대한 그래디언트를 계산
        optimizer.step() # 계산된 그래디언트를 사용하여 모델의 파라미터를 업데이트함
        running_loss += loss.item()

    # 검증 단계
    val_loss = 0.0
    model.eval() # 모델을 평가 모드로 설정 - 드롭아웃 등의 레이어가 비활성화  됨
    with torch.no_grad(): # 검증 단계에서 그래디언트 계산을 비활성화하여 메모리 사용량을 줄임
        for inputs, labels in valloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(valloader) # 각 에폭에서 검증 데이터셋의 평균 소실을 계산
    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(trainloader)}, Val Loss: {val_loss}')

    # 검증 손실이 개선되었는지 확인하고 모델 저장
    if val_loss < best_val_loss:
        print(f'Validation Loss Decreased ({best_val_loss: .6f} ---> {val_loss:-.6f}) \t Saving The Model')
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), 'best_model.pth') # 모델 저장
    else:
        trials += 1
        if trials >= patience: # 조기 종료 조건 충족 확인 - 검증 손실이 개선되지 않으면 trials를 증가시키고, patience를 초과하면 조기 종료함
            print("Early stopping triggered")
            break

# 최고의 모델을 불러와서 평가
model.load_state_dict(torch.load('best_model.pth')) # 모델 상태 불러오기 (가장 성능이 좋은 모델 파라미터 불러오기)

# 모델 평가
correct = 0
total = 0
with torch.no_grad(): # 그래디언트 계산 비활성화
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1) # torch.max(input, dim), 'dim'은 차원을 의미. 클래스 점수들 중에서 최대값을 찾는다는 의미
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100*correct / total # 정확도 계산

print(f"Accuracy on the 10000 test images: {accuracy}%") # 정확도 출력

Epoch 1, Train Loss: 0.2391227756695201, Val Loss: 0.09077995874364167
Validation Loss Decreased ( inf ---> 0.090780) 	 Saving The Model
Epoch 2, Train Loss: 0.06879109606084724, Val Loss: 0.05989667178042471
Validation Loss Decreased ( 0.090780 ---> 0.059897) 	 Saving The Model
Epoch 3, Train Loss: 0.04857162973626206, Val Loss: 0.06416875039390783
Epoch 4, Train Loss: 0.039116560633604726, Val Loss: 0.05574730338536183
Validation Loss Decreased ( 0.059897 ---> 0.055747) 	 Saving The Model
Epoch 5, Train Loss: 0.030650813326317194, Val Loss: 0.05728038072033706
Epoch 6, Train Loss: 0.02629721337122222, Val Loss: 0.05719768921312123
Epoch 7, Train Loss: 0.021861348884917486, Val Loss: 0.05565781458194288
Validation Loss Decreased ( 0.055747 ---> 0.055658) 	 Saving The Model
Epoch 8, Train Loss: 0.016673741163084437, Val Loss: 0.051332603487683995
Validation Loss Decreased ( 0.055658 ---> 0.051333) 	 Saving The Model
Epoch 9, Train Loss: 0.014773651492219262, Val Loss: 0.050648287690567

<ipython-input-3-bad5ee9e5e07>:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth')) # 모델 상태 불러오기


Accuracy on the 10000 test images: 98.88%


- torch: 이것은 메인 PyTorch 라이브러리입니다. 여기에는 GPU를 통한 가속 텐서 계산 지원, 신경망 훈련을 용이하게 하는 자동 차별화, 모델 구축 및 훈련을 위한 다양한 유틸리티가 포함됩니다.
- torch.nn: 레이어, 활성화 함수, 손실 함수와 같은 신경망의 구성 요소를 제공하는 PyTorch의 하위 모듈입니다. 신경망의 아키텍처를 정의하는 데 필수적입니다.
- torch.nn.function: 이 모듈에는 torch.nn 레이어에서 사용되는 기능이 포함되어 있습니다. 입력 데이터 및 가중치에 이러한 함수를 직접 사용할 수 있으므로 일부 작업에 더 많은 유연성을 제공합니다. 여기에는 활성화, 손실 계산 및 상태(즉, 가중치)를 유지하지 않는 다양한 기타 작업을 위한 함수가 포함됩니다.
- torch.optim: 이 하위 모듈은 SGD(Stochastic Gradient Descent), Adam 등과 같은 신경망 훈련을 위한 최적화 알고리즘을 제공합니다. 이러한 최적화 프로그램은 계산된 기울기를 기반으로 네트워크의 가중치를 업데이트하는 데 사용됩니다.
- torchvision: 이미지 데이터 작업을 위한 유틸리티를 제공하는 PyTorch 프로젝트의 패키지입니다. 여기에는 사전 정의된 데이터세트(예: MNIST, CIFAR10, FashionMNIST), 모델 아키텍처(예: ResNet, AlexNet) 및 전처리를 위한 일반적인 이미지 변환이 포함됩니다.
- torchvision.transforms: 일반적인 이미지 변환을 제공하는 torchvision 내의 모듈입니다. 이는 이미지를 신경망에 공급하기 전에 데이터 증대 및 이미지 전처리에 사용될 수 있습니다. 예로는 크기 조정, 정규화, 텐서로 변환 등이 있습니다.
- SubsetRandomSampler: 교체 없이 데이터세트에서 요소를 무작위로 샘플링하는 데 사용되는 도구입니다. 데이터 세트를 훈련 및 검증/테스트 세트로 분할하거나 모델 훈련을 위해 사용자 정의 데이터 샘플링 전략을 구현하려는 경우에 특히 유용

Q. PyTorch를 사용하여 FashionMNIST 데이터세트에 대한 분류 모델링 및 평가를 다음과 같은 단계로 수행하세요.
- 1단계: 신경망 모델 정의 - Conv layer 추가
- 2단계: FashionMNIST 데이터셋 로드
- 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
- 4단계: 조기 종료를 포함한 모델 학습 및 best model 저장
- 5단계: best model을 로드하고 테스트 데이터셋으로 평가

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision # torchvision에서 데이터 가져올 예정
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.transforms import ToTensor

In [ ]:
# 1. 데이터셋 전처리 과정 정의
# Fashion.MNIST 데이터셋을 위한 전처리 과정 정의
transform=transforms.Compose([
    transforms.ToTensor(), # 이미지를 PyTorch 텐서로 변환
    transforms.Normalize((0.5, ), (0.5, )) # 이미지를 평균 0.5, 표준편차 0.5로 정규화하여 [-1, 1] 범위로 조정
])


In [ ]:
# 2. Fashion.MNIST 데이터 로드
train_dataset=torchvision.datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_dataset=torchvision.datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

100%|██████████| 26421880/26421880 [00:06<00:00, 4025122.41it/s] 


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 284365.64it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5130965.57it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 15601356.21it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
# 데이터셋 확인
train_dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [ ]:
!ls data

FashionMNIST


In [ ]:
# 3. 데이터셋을 훈련 및 검증 세트로 분할
# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size=int(0.8*len(train_dataset))
val_size=len(train_dataset)-train_size
train_dataset, val_dataset=random_split(train_dataset, [train_size, val_size])

In [ ]:
# 4. 데이터로더를 생성
trainloader=DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader=DataLoader(val_dataset, batch_size=64, shuffle=False)
testloader=DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# 5. 모델 아키텍쳐 정의 (각 레이어에 대한 정의 - 클래스 활용)
# 모델 정의:
class FashionMNIST_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,20,5)
        self.pool=nn.MaxPool2d(2,2) # 컨볼루션 신경망(CNN)에 매우 중요함. 공간적 차원 축소를 통해 데이터 크기를 줄여 연산 비용 감소 및 모델의 학습과 추론 속도를 향상시킴
        self.flatten=nn.Flatten() # 컨볼루션 레이어와 풀링 레이어는  2D 혹은 3D 데이터를 입력 받는데 '완전 연결 레이어 (nn.Linear)'는 1차원 입력을 기대함. 따라서, flatten을 통해 다차원 데이터를 1차원으로 변환해 완전 연결 레이어에 전달해줌
        self.fc1=nn.Linear(2880, 50) # 완전 연결 레이어.
        self.fc2=nn.Linear(50,10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # ReLU 활성화 함수 적용 후 맥스풀링
        x = self.flatten(x) # 평탄화
        x = F.relu(self.fc1(x)) # ReLU 활성화 함수 적용
        x = self.fc2(x) # 최종 출력
        return x

# 모델을 정의하고 장치에 할당
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model= FashionMNIST_model().to(device)
print(model)

FashionMNIST_model(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2880, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [ ]:
# 손실 함수 및 최적화 알고리즘 지정
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
# 모델 훈련 및 검증
best_val_loss = float('inf') # 검증 손실을 추적하기 위한 변수 초기화
patience, trials = 5, 0 #  조기 종료 기준 설정
num_epochs = 20 # 에폭 수 설정
for epoch in range(num_epochs):
    model.train() # 모델을 훈련 모드로 설정
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad() # 이전 배치의 그래디언트를 초기화
        outputs = model(inputs) # 모델을 통한 순전파를 수행하여 예측 값을 얻기
        loss = criterion(outputs, labels) # 손실 계산
        loss.backward() # 역전파. 손실에 대한 그래디언트를 계산
        optimizer.step() # 계산된 그래디언트를 사용하여 모델의 파라미터를 업데이트함
        running_loss += loss.item()

    # 검증 단계
    val_loss = 0.0
    model.eval() # 모델을 평가 모드로 설정 - 드롭아웃 등의 레이어가 비활성화  됨
    with torch.no_grad(): # 검증 단계에서 그래디언트 계산을 비활성화하여 메모리 사용량을 줄임
        for inputs, labels in valloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(valloader) # 각 에폭에서 검증 데이터셋의 평균 소실을 계산
    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(trainloader)}, Val Loss: {val_loss}')

    # 검증 손실이 개선되었는지 확인하고 모델 저장
    if val_loss < best_val_loss:
        print(f'Validation Loss Decreased ({best_val_loss: .6f} ---> {val_loss:-.6f}) \t Saving The Model')
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), 'best_model.pth') # 모델 저장
    else:
        trials += 1
        if trials >= patience: # 조기 종료 조건 충족 확인 - 검증 손실이 개선되지 않으면 trials를 증가시키고, patience를 초과하면 조기 종료함
            print("Early stopping triggered")
            break

# 최고의 모델을 불러와서 평가
model.load_state_dict(torch.load('best_model.pth')) # 모델 상태 불러오기 (가장 성능이 좋은 모델 파라미터 불러오기)

Epoch 1, Train Loss: 0.5345916992425919, Val Loss: 0.3777507017267511
Validation Loss Decreased ( inf ---> 0.377751) 	 Saving The Model
Epoch 2, Train Loss: 0.34076169367631276, Val Loss: 0.3188100238429739
Validation Loss Decreased ( 0.377751 ---> 0.318810) 	 Saving The Model
Epoch 3, Train Loss: 0.29901945814490316, Val Loss: 0.3002368935007364
Validation Loss Decreased ( 0.318810 ---> 0.300237) 	 Saving The Model
Epoch 4, Train Loss: 0.2658626327216625, Val Loss: 0.26215183623927707
Validation Loss Decreased ( 0.300237 ---> 0.262152) 	 Saving The Model
Epoch 5, Train Loss: 0.2423629157692194, Val Loss: 0.28056232349828203
Epoch 6, Train Loss: 0.2254350418895483, Val Loss: 0.2553574903174601
Validation Loss Decreased ( 0.262152 ---> 0.255357) 	 Saving The Model
Epoch 7, Train Loss: 0.20687655357519785, Val Loss: 0.2708371213458954
Epoch 8, Train Loss: 0.19595322700341541, Val Loss: 0.24926589655273773
Validation Loss Decreased ( 0.255357 ---> 0.249266) 	 Saving The Model
Epoch 9, Tra

<ipython-input-15-d1426db49763>:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth')) # 모델 상태 불러오기 (가장 성능이 좋은 모델 파라미터 불러오기)


<All keys matched successfully>

In [ ]:
# 모델 평가
correct = 0
total = 0
with torch.no_grad(): # 그래디언트 계산 비활성화
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1) # torch.max(input, dim), 'dim'은 차원을 의미. 클래스 점수들 중에서 최대값을 찾는다는 의미
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100*correct / total # 정확도 계산

print(f"Accuracy on the 10000 test images: {accuracy}%") # 정확도 출력

Accuracy on the 10000 test images: 89.97%


#### 강사님의 FashionMNIST

###### conv 사용하지 않은 버전

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

# 1단계: 신경망 모델 정의
# Net 클래스는 nn.Module을 상속받아 만들어진 사용자 정의 신경망 모델로, FashionMNIST 데이터셋의 이미지 분류를 위해 설계
class Net(nn.Module):
# 모델의 구조를 정의합니다. 이 모델은 세 개의 완전 연결(fully-connected) 레이어를 포함
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 120)  # 입력으로 28x28 크기의 이미지를 받아 펼친 후(784개의 피처), 120개의 출력 노드로 연결
        self.fc2 = nn.Linear(120, 84) # 120개의 입력 노드에서 84개의 출력 노드로 연결
        self.fc3 = nn.Linear(84, 10)  # 84개의 입력 노드에서 최종적으로 10개의 출력 노드로 연결
# forward 메소드는 신경망에 데이터를 어떻게 전달할지 정의
    def forward(self, x):
        x = x.view(-1, 28 * 28) # 입력 이미지 x를 1차원으로 펼치는 과정입니다. -1은 배치 크기에 대응되며, 이는 어떤 크기의 배치라도 처리할 수 있음을 의미
        x = F.relu(self.fc1(x)) # 첫 번째 레이어를 통과한 후, ReLU 활성화 함수를 적용합니다. 이는 비선형성을 도입하여 모델이 더 복잡한 패턴을 학습
        x = F.relu(self.fc2(x)) # 두 번째 레이어를 통과한 후, 다시 ReLU 활성화 함수를 적용
        x = self.fc3(x) # 세 번째 레이어를 통과한 결과를 반환. 소프트맥스 함수 등을 통해 확률로 변환되어 클래스를 예측하는 데 사용
        return x

# 2단계: FashionMNIST 데이터셋 로드
# torchvision 라이브러리를 사용하여 FashionMNIST 데이터셋을 다운로드하고, 데이터를 전처리하기 위한 변환(transform)을 설정하는 과정

# transforms.Compose는 여러 전처리 단계를 하나로 묶어주는 역할
# transforms.ToTensor(): 이미지를 PyTorch 텐서로 변환. 이미지의 픽셀 값 범위가 0에서 255 사이의 정수에서 0.0에서 1.0 사이의 부동소수점으로 변경
# 모든 채널의 평균을 0.5로, 표준편차를 0.5로 설정합니다. 이는 데이터의 범위를 대략적으로 -1.0에서 1.0 사이로 조정
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# FashionMNIST 데이터셋을 다운로드하고, 지정된 변환을 적용하여 데이터를 준비하는 함수
train_val_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 및 검증 분할을 위한 데이터 인덱스 생성
dataset_size = len(train_val_dataset) # 훈련 및 검증 데이터셋의 전체 크기
indices = list(range(dataset_size)) # 데이터셋 내의 모든 샘플에 대한 인덱스를 포함
validation_split = 0.1 # 검증 세트로 사용될 데이터의 비율
split = int(np.floor(validation_split * dataset_size)) # 검증 세트의 크기를 계산
np.random.shuffle(indices) # 훈련 및 검증 세트가 데이터셋의 특정 부분에 치우치지 않도록 하기 위함
train_indices, val_indices = indices[split:], indices[:split] # 섞인 인덱스를 사용하여 훈련 세트와 검증 세트의 인덱스를 분할

# PT 데이터 샘플러 및 로더 생성
# 훈련 세트와 검증 세트에 대한 데이터 로더를 설정하고, 테스트 세트에 대한 데이터 로더를 별도로 설정하는 과정입니다.
# 이러한 데이터 로더들은 모델 학습, 검증, 테스트 과정에서 배치 단위로 데이터를 로드하는 데 사용
train_sampler = SubsetRandomSampler(train_indices) # train_indices에 해당하는 훈련 데이터의 인덱스를 무작위로 샘플링하는 샘플러를 생성
val_sampler = SubsetRandomSampler(val_indices) # val_indices에 해당하는 검증 데이터의 인덱스로부터 데이터를 무작위로 샘플링하는 샘플러를 생성
# 배치 크기 4로 로드하는 훈련 데이터 로더를 생성(# 데이터의 무작위 샘플링을 수행)
trainloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=train_sampler)
valloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=val_sampler)
# shuffle=False 인자를 통해 셔플링 없이 순서대로 데이터를 로드. 테스트 과정에서는 데이터의 순서가 결과에 영향을 미치지 않으므로 셔플링을 수행하지 않습니다.
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

# 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
net = Net() #  클래스의 인스턴스를 생성하여 net 변수에 할당
criterion = nn.CrossEntropyLoss() # 멀티클래스 분류 문제에서 널리 사용되는 손실 함수로, 모델의 예측값과 실제 타겟값 사이의 차이를 측정
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # 모멘텀은 최적화 과정에서 이전 그래디언트의 방향을 고려, 파라미터 업데이트 시 관성을 부여

# 4단계: 조기 종료를 포함한 모델 학습
patience = 5 # 검증 손실이 개선되지 않을 때, 훈련을 계속 진행하기 전에 기다릴 에폭 수를 의미
patience_counter = 0
best_val_loss = np.Inf # 최고의 검증 손실 값을 무한대로 초기화. 훈련 과정에서 검증 손실이 이전에 기록된 최소 손실보다 낮아지면 업데이트되는 값

# 모델을 에폭 단위로 반복 훈련시키면서, 각 배치의 손실을 계산하고 모델 파라미터를 업데이트하는 기본적인 훈련 과정을 구현
for epoch in range(20):  # 데이터셋을 여러 번 반복
    net.train()  # 모델을 학습 모드로 설정. 이는 모델 내의 특정 레이어(예: 드롭아웃, 배치 정규화 등)가 훈련 시와 평가 시 다르게 동작해야 할 때 필요
    running_loss = 0.0 # 현재 에폭의 총 손실을 계산하기 위해 실행 손실을 0으로 초기화
    for i, data in enumerate(trainloader, 0): # '0'은 열거의 시작 인덱스를 지정
        inputs, labels = data
        optimizer.zero_grad() # 최적화를 수행하기 전에 모델의 그래디언트를 0으로 초기화
        outputs = net(inputs) # 현재 배치의 입력 데이터를 모델에 전달하여 예측값을 계산
        loss = criterion(outputs, labels) # 모델의 예측값과 실제 레이블 간의 손실을 계산
        loss.backward() # 손실 함수의 그래디언트를 역전파합니다. 이 과정에서 모델 파라미터에 대한 손실의 미분값이 계산
        optimizer.step() # 계산된 그래디언트를 사용하여 모델의 파라미터를 업데이트
        running_loss += loss.item() # 현재 배치의 손실을 실행 손실에 누적. 이를 통해 전체 에폭의 평균 손실을 계산

    # 검증 단계
    net.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0 # 검증 손실을 계산하기 위한 변수를 0으로 초기화
    with torch.no_grad(): # 이 블록 내에서는 그래디언트 계산을 비활성화. 평가 모드에서는 모델을 업데이트하지 않으므로, 그래디언트를 계산할 필요가 없습니다
        for inputs, labels in valloader: # 검증 데이터 로더(valloader)에서 배치 단위로 데이터를 로드하여 반복
            outputs = net(inputs)
            loss = criterion(outputs, labels) # criterion은 손실 함수로, 모델의 성능을 측정하는 기준
            val_loss += loss.item() # 누적된 검증 손실을 검증 데이터 배치의 총 수로 나누어 평균 검증 손실을 계산
    running_loss /= len(trainloader)
    val_loss /= len(valloader)
    print(f'에폭 {epoch + 1}, 훈련 손실: {running_loss}, 검증 손실: {val_loss}')

    # 조기 종료 체크
    if val_loss < best_val_loss: # 현재 에폭에서 계산된 검증 손실(val_loss)이 이전에 기록된 최소 검증 손실(best_val_loss)보다 낮은지 확인
        print(f'검증 손실이 감소하였습니다. ({best_val_loss:.6f} 에서 {val_loss:.6f}로). 모델 저장 중...')
        torch.save(net.state_dict(), '/content/drive/MyDrive/KDT_240424/m6_DL/data/model/best_model.pth')
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('조기 종료 발동.')
            break

# 5단계: 최고의 모델을 로드하고 테스트 데이터셋으로 평가
# torch.load 함수는 지정된 경로에서 모델 파라미터를 불러오며, load_state_dict 메서드를 사용하여 이 파라미터를 현재 net 모델에 로드
net.load_state_dict(torch.load('/content/drive/MyDrive/KDT_240424/m6_DL/data/model/best_model.pth'))
correct = 0 # 정확히 분류된 샘플의 수를 세기 위한 변수
total = 0 # 테스트셋의 전체 샘플 수를 세기 위한 변수
with torch.no_grad():
    for data in testloader: # 테스트 데이터셋을 배치 단위로 순회
        images, labels = data
        outputs = net(images) # 현재 배치의 이미지를 모델에 전달하여 예측값을 계산
        _, predicted = torch.max(outputs.data, 1) # torch.max는 각 예측에 대한 최대값과 그 위치(인덱스)를 반환. 위치만 필요하므로 _를 사용하여 최대값은 무시
        total += labels.size(0) # labels.size(0)는 현재 배치의 크기(샘플 수)
        correct += (predicted == labels).sum().item() # 일치하는 경우의 수를 텐서 형태로 반환하며, .item()으로 이를 파이썬의 스칼라 값으로 변환

print(f'10000개의 테스트 이미지에 대한 네트워크의 정확도: {100 * correct / total} %')



에폭 1, 훈련 손실: 0.535651011273377, 검증 손실: 0.40922316518542357
검증 손실이 감소하였습니다. (inf 에서 0.409223로). 모델 저장 중...
에폭 2, 훈련 손실: 0.39227010604070756, 검증 손실: 0.3866698606573821
검증 손실이 감소하였습니다. (0.409223 에서 0.386670로). 모델 저장 중...
에폭 3, 훈련 손실: 0.3533285798632423, 검증 손실: 0.3568271454299538
검증 손실이 감소하였습니다. (0.386670 에서 0.356827로). 모델 저장 중...
에폭 4, 훈련 손실: 0.32614926484557516, 검증 손실: 0.3376264184047565
검증 손실이 감소하였습니다. (0.356827 에서 0.337626로). 모델 저장 중...
에폭 5, 훈련 손실: 0.3067896236815489, 검증 손실: 0.32901990392623703
검증 손실이 감소하였습니다. (0.337626 에서 0.329020로). 모델 저장 중...
에폭 6, 훈련 손실: 0.29086293218971465, 검증 손실: 0.33687869480469573
에폭 7, 훈련 손실: 0.2778573052465281, 검증 손실: 0.3390671833789917
에폭 8, 훈련 손실: 0.26600797306730295, 검증 손실: 0.32028469232229306
검증 손실이 감소하였습니다. (0.329020 에서 0.320285로). 모델 저장 중...
에폭 9, 훈련 손실: 0.25611929553997254, 검증 손실: 0.33395229578492097
에폭 10, 훈련 손실: 0.24738100272719665, 검증 손실: 0.32434816871866434
에폭 11, 훈련 손실: 0.23838482663386612, 검증 손실: 0.3229594364578516
에폭 12, 훈련 손실: 0.22751199383366

<ipython-input-20-c0080c9348ce>:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('/content/drive/MyDrive/KDT_240424/m6_DL/data/model/best_mo

10000개의 테스트 이미지에 대한 네트워크의 정확도: 88.46 %


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

# 1단계: 신경망 모델 정의
# Net 클래스는 nn.Module을 상속받아 만들어진 사용자 정의 신경망 모델로, FashionMNIST 데이터셋의 이미지 분류를 위해 설계
class Net(nn.Module):
# 모델의 구조를 정의합니다. 이 모델은 세 개의 완전 연결(fully-connected) 레이어를 포함
    def __init__(self):
        super(Net, self).__init__()
        # 첫 번째 합성곱 레이어: 입력 채널 1개, 출력 채널 6개, 커널 크기 5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 두 번째 합성곱 레이어: 입력 채널 6개, 출력 채널 16개, 커널 크기 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 전결합 레이어
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 16개의 채널과 4x4 이미지 크기
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 첫 번째 합성곱, ReLU 활성화, 맥스 풀링
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        # 두 번째 합성곱, ReLU 활성화, 맥스 풀링
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        # 입력 데이터를 1차원으로 펼침 (flatten 대신 view로 1차원으로 차원 축소)
        x = x.view(-1, 16 * 4 * 4)
        # 전결합 레이어와 ReLU 활성화 함수 적용
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 마지막 레이어 출력
        x = self.fc3(x)
        return x

# 2단계: FashionMNIST 데이터셋 로드
# torchvision 라이브러리를 사용하여 FashionMNIST 데이터셋을 다운로드하고, 데이터를 전처리하기 위한 변환(transform)을 설정하는 과정

# transforms.Compose는 여러 전처리 단계를 하나로 묶어주는 역할
# transforms.ToTensor(): 이미지를 PyTorch 텐서로 변환. 이미지의 픽셀 값 범위가 0에서 255 사이의 정수에서 0.0에서 1.0 사이의 부동소수점으로 변경
# 모든 채널의 평균을 0.5로, 표준편차를 0.5로 설정합니다. 이는 데이터의 범위를 대략적으로 -1.0에서 1.0 사이로 조정
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# FashionMNIST 데이터셋을 다운로드하고, 지정된 변환을 적용하여 데이터를 준비하는 함수
train_val_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 및 검증 분할을 위한 데이터 인덱스 생성
dataset_size = len(train_val_dataset) # 훈련 및 검증 데이터셋의 전체 크기
indices = list(range(dataset_size)) # 데이터셋 내의 모든 샘플에 대한 인덱스를 포함
validation_split = 0.1 # 검증 세트로 사용될 데이터의 비율
split = int(np.floor(validation_split * dataset_size)) # 검증 세트의 크기를 계산
np.random.shuffle(indices) # 훈련 및 검증 세트가 데이터셋의 특정 부분에 치우치지 않도록 하기 위함
train_indices, val_indices = indices[split:], indices[:split] # 섞인 인덱스를 사용하여 훈련 세트와 검증 세트의 인덱스를 분할

# PT 데이터 샘플러 및 로더 생성
# 훈련 세트와 검증 세트에 대한 데이터 로더를 설정하고, 테스트 세트에 대한 데이터 로더를 별도로 설정하는 과정입니다.
# 이러한 데이터 로더들은 모델 학습, 검증, 테스트 과정에서 배치 단위로 데이터를 로드하는 데 사용
train_sampler = SubsetRandomSampler(train_indices) # train_indices에 해당하는 훈련 데이터의 인덱스를 무작위로 샘플링하는 샘플러를 생성
val_sampler = SubsetRandomSampler(val_indices) # val_indices에 해당하는 검증 데이터의 인덱스로부터 데이터를 무작위로 샘플링하는 샘플러를 생성
# 배치 크기 4로 로드하는 훈련 데이터 로더를 생성(# 데이터의 무작위 샘플링을 수행)
trainloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=train_sampler)
valloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=val_sampler)
# shuffle=False 인자를 통해 셔플링 없이 순서대로 데이터를 로드. 테스트 과정에서는 데이터의 순서가 결과에 영향을 미치지 않으므로 셔플링을 수행하지 않습니다.
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

# 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
net = Net() #  클래스의 인스턴스를 생성하여 net 변수에 할당
criterion = nn.CrossEntropyLoss() # 멀티클래스 분류 문제에서 널리 사용되는 손실 함수로, 모델의 예측값과 실제 타겟값 사이의 차이를 측정
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # 모멘텀은 최적화 과정에서 이전 그래디언트의 방향을 고려, 파라미터 업데이트 시 관성을 부여

# 4단계: 조기 종료를 포함한 모델 학습
patience = 5 # 검증 손실이 개선되지 않을 때, 훈련을 계속 진행하기 전에 기다릴 에폭 수를 의미
patience_counter = 0
best_val_loss = np.Inf # 최고의 검증 손실 값을 무한대로 초기화. 훈련 과정에서 검증 손실이 이전에 기록된 최소 손실보다 낮아지면 업데이트되는 값

# 모델을 에폭 단위로 반복 훈련시키면서, 각 배치의 손실을 계산하고 모델 파라미터를 업데이트하는 기본적인 훈련 과정을 구현
for epoch in range(20):  # 데이터셋을 여러 번 반복
    net.train()  # 모델을 학습 모드로 설정. 이는 모델 내의 특정 레이어(예: 드롭아웃, 배치 정규화 등)가 훈련 시와 평가 시 다르게 동작해야 할 때 필요
    running_loss = 0.0 # 현재 에폭의 총 손실을 계산하기 위해 실행 손실을 0으로 초기화
    for i, data in enumerate(trainloader, 0): # '0'은 열거의 시작 인덱스를 지정
        inputs, labels = data
        optimizer.zero_grad() # 최적화를 수행하기 전에 모델의 그래디언트를 0으로 초기화
        outputs = net(inputs) # 현재 배치의 입력 데이터를 모델에 전달하여 예측값을 계산
        loss = criterion(outputs, labels) # 모델의 예측값과 실제 레이블 간의 손실을 계산
        loss.backward() # 손실 함수의 그래디언트를 역전파합니다. 이 과정에서 모델 파라미터에 대한 손실의 미분값이 계산
        optimizer.step() # 계산된 그래디언트를 사용하여 모델의 파라미터를 업데이트
        running_loss += loss.item() # 현재 배치의 손실을 실행 손실에 누적. 이를 통해 전체 에폭의 평균 손실을 계산

    # 검증 단계
    net.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0 # 검증 손실을 계산하기 위한 변수를 0으로 초기화
    with torch.no_grad(): # 이 블록 내에서는 그래디언트 계산을 비활성화. 평가 모드에서는 모델을 업데이트하지 않으므로, 그래디언트를 계산할 필요가 없습니다
        for inputs, labels in valloader: # 검증 데이터 로더(valloader)에서 배치 단위로 데이터를 로드하여 반복
            outputs = net(inputs)
            loss = criterion(outputs, labels) # criterion은 손실 함수로, 모델의 성능을 측정하는 기준
            val_loss += loss.item() # 누적된 검증 손실을 검증 데이터 배치의 총 수로 나누어 평균 검증 손실을 계산
    running_loss /= len(trainloader)
    val_loss /= len(valloader)
    print(f'에폭 {epoch + 1}, 훈련 손실: {running_loss}, 검증 손실: {val_loss}')

    # 조기 종료 체크
    if val_loss < best_val_loss: # 현재 에폭에서 계산된 검증 손실(val_loss)이 이전에 기록된 최소 검증 손실(best_val_loss)보다 낮은지 확인
        print(f'검증 손실이 감소하였습니다. ({best_val_loss:.6f} 에서 {val_loss:.6f}로). 모델 저장 중...')
        torch.save(net.state_dict(), '/content/drive/MyDrive/kdt_240424/m6_dl/data/model/best_model.pth')
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('조기 종료 발동.')
            break

# 5단계: 최고의 모델을 로드하고 테스트 데이터셋으로 평가
# torch.load 함수는 지정된 경로에서 모델 파라미터를 불러오며, load_state_dict 메서드를 사용하여 이 파라미터를 현재 net 모델에 로드
net.load_state_dict(torch.load('/content/drive/MyDrive/kdt_240424/m6_dl/data/model/best_model.pth'))
correct = 0 # 정확히 분류된 샘플의 수를 세기 위한 변수
total = 0 # 테스트셋의 전체 샘플 수를 세기 위한 변수
with torch.no_grad():
    for data in testloader: # 테스트 데이터셋을 배치 단위로 순회
        images, labels = data
        outputs = net(images) # 현재 배치의 이미지를 모델에 전달하여 예측값을 계산
        _, predicted = torch.max(outputs.data, 1) # torch.max는 각 예측에 대한 최대값과 그 위치(인덱스)를 반환. 위치만 필요하므로 _를 사용하여 최대값은 무시
        total += labels.size(0) # labels.size(0)는 현재 배치의 크기(샘플 수)
        correct += (predicted == labels).sum().item() # 일치하는 경우의 수를 텐서 형태로 반환하며, .item()으로 이를 파이썬의 스칼라 값으로 변환

print(f'10000개의 테스트 이미지에 대한 네트워크의 정확도: {100 * correct / total} %')



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

# 1단계: 신경망 모델 정의
# Net 클래스는 nn.Module을 상속받아 만들어진 사용자 정의 신경망 모델로, FashionMNIST 데이터셋의 이미지 분류를 위해 설계
class Net(nn.Module):
# 모델의 구조를 정의합니다. 이 모델은 세 개의 완전 연결(fully-connected) 레이어를 포함
    def __init__(self):
        super(Net, self).__init__()
        # 첫 번째 합성곱 레이어: 입력 채널 1개, 출력 채널 6개, 커널 크기 5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 두 번째 합성곱 레이어: 입력 채널 6개, 출력 채널 16개, 커널 크기 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2,2)
        # 전결합 레이어
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 16개의 채널과 4x4 이미지 크기
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 첫 번째 합성곱, ReLU 활성화, 맥스 풀링
        x = self.pool(F.relu(self.conv1(x)))
        # 두 번째 합성곱, ReLU 활성화, 맥스 풀링
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        # 입력 데이터를 1차원으로 펼침 (flatten 대신 view로 1차원으로 차원 축소)
        # 전결합 레이어와 ReLU 활성화 함수 적용
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # 마지막 레이어 출력
        x = self.fc3(x)
        return x

# 2단계: FashionMNIST 데이터셋 로드
# torchvision 라이브러리를 사용하여 FashionMNIST 데이터셋을 다운로드하고, 데이터를 전처리하기 위한 변환(transform)을 설정하는 과정

# transforms.Compose는 여러 전처리 단계를 하나로 묶어주는 역할
# transforms.ToTensor(): 이미지를 PyTorch 텐서로 변환. 이미지의 픽셀 값 범위가 0에서 255 사이의 정수에서 0.0에서 1.0 사이의 부동소수점으로 변경
# 모든 채널의 평균을 0.5로, 표준편차를 0.5로 설정합니다. 이는 데이터의 범위를 대략적으로 -1.0에서 1.0 사이로 조정
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# FashionMNIST 데이터셋을 다운로드하고, 지정된 변환을 적용하여 데이터를 준비하는 함수
train_val_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 및 검증 분할을 위한 데이터 인덱스 생성
dataset_size = len(train_val_dataset) # 훈련 및 검증 데이터셋의 전체 크기
indices = list(range(dataset_size)) # 데이터셋 내의 모든 샘플에 대한 인덱스를 포함
validation_split = 0.1 # 검증 세트로 사용될 데이터의 비율
split = int(np.floor(validation_split * dataset_size)) # 검증 세트의 크기를 계산
np.random.shuffle(indices) # 훈련 및 검증 세트가 데이터셋의 특정 부분에 치우치지 않도록 하기 위함
train_indices, val_indices = indices[split:], indices[:split] # 섞인 인덱스를 사용하여 훈련 세트와 검증 세트의 인덱스를 분할

# PT 데이터 샘플러 및 로더 생성
# 훈련 세트와 검증 세트에 대한 데이터 로더를 설정하고, 테스트 세트에 대한 데이터 로더를 별도로 설정하는 과정입니다.
# 이러한 데이터 로더들은 모델 학습, 검증, 테스트 과정에서 배치 단위로 데이터를 로드하는 데 사용
train_sampler = SubsetRandomSampler(train_indices) # train_indices에 해당하는 훈련 데이터의 인덱스를 무작위로 샘플링하는 샘플러를 생성
val_sampler = SubsetRandomSampler(val_indices) # val_indices에 해당하는 검증 데이터의 인덱스로부터 데이터를 무작위로 샘플링하는 샘플러를 생성
# 배치 크기 4로 로드하는 훈련 데이터 로더를 생성(# 데이터의 무작위 샘플링을 수행)
trainloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=train_sampler)
valloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=val_sampler)
# shuffle=False 인자를 통해 셔플링 없이 순서대로 데이터를 로드. 테스트 과정에서는 데이터의 순서가 결과에 영향을 미치지 않으므로 셔플링을 수행하지 않습니다.
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

# 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
net = Net() #  클래스의 인스턴스를 생성하여 net 변수에 할당
criterion = nn.CrossEntropyLoss() # 멀티클래스 분류 문제에서 널리 사용되는 손실 함수로, 모델의 예측값과 실제 타겟값 사이의 차이를 측정
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # 모멘텀은 최적화 과정에서 이전 그래디언트의 방향을 고려, 파라미터 업데이트 시 관성을 부여

# 4단계: 조기 종료를 포함한 모델 학습
patience = 5 # 검증 손실이 개선되지 않을 때, 훈련을 계속 진행하기 전에 기다릴 에폭 수를 의미
patience_counter = 0
best_val_loss = np.Inf # 최고의 검증 손실 값을 무한대로 초기화. 훈련 과정에서 검증 손실이 이전에 기록된 최소 손실보다 낮아지면 업데이트되는 값

# 모델을 에폭 단위로 반복 훈련시키면서, 각 배치의 손실을 계산하고 모델 파라미터를 업데이트하는 기본적인 훈련 과정을 구현
for epoch in range(20):  # 데이터셋을 여러 번 반복
    net.train()  # 모델을 학습 모드로 설정. 이는 모델 내의 특정 레이어(예: 드롭아웃, 배치 정규화 등)가 훈련 시와 평가 시 다르게 동작해야 할 때 필요
    running_loss = 0.0 # 현재 에폭의 총 손실을 계산하기 위해 실행 손실을 0으로 초기화
    for i, data in enumerate(trainloader, 0): # '0'은 열거의 시작 인덱스를 지정
        inputs, labels = data
        optimizer.zero_grad() # 최적화를 수행하기 전에 모델의 그래디언트를 0으로 초기화
        outputs = net(inputs) # 현재 배치의 입력 데이터를 모델에 전달하여 예측값을 계산
        loss = criterion(outputs, labels) # 모델의 예측값과 실제 레이블 간의 손실을 계산
        loss.backward() # 손실 함수의 그래디언트를 역전파합니다. 이 과정에서 모델 파라미터에 대한 손실의 미분값이 계산
        optimizer.step() # 계산된 그래디언트를 사용하여 모델의 파라미터를 업데이트
        running_loss += loss.item() # 현재 배치의 손실을 실행 손실에 누적. 이를 통해 전체 에폭의 평균 손실을 계산

    # 검증 단계
    net.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0 # 검증 손실을 계산하기 위한 변수를 0으로 초기화
    with torch.no_grad(): # 이 블록 내에서는 그래디언트 계산을 비활성화. 평가 모드에서는 모델을 업데이트하지 않으므로, 그래디언트를 계산할 필요가 없습니다
        for inputs, labels in valloader: # 검증 데이터 로더(valloader)에서 배치 단위로 데이터를 로드하여 반복
            outputs = net(inputs)
            loss = criterion(outputs, labels) # criterion은 손실 함수로, 모델의 성능을 측정하는 기준
            val_loss += loss.item() # 누적된 검증 손실을 검증 데이터 배치의 총 수로 나누어 평균 검증 손실을 계산
    running_loss /= len(trainloader)
    val_loss /= len(valloader)
    print(f'에폭 {epoch + 1}, 훈련 손실: {running_loss}, 검증 손실: {val_loss}')

    # 조기 종료 체크
    if val_loss < best_val_loss: # 현재 에폭에서 계산된 검증 손실(val_loss)이 이전에 기록된 최소 검증 손실(best_val_loss)보다 낮은지 확인
        print(f'검증 손실이 감소하였습니다. ({best_val_loss:.6f} 에서 {val_loss:.6f}로). 모델 저장 중...')
        torch.save(net.state_dict(), '/content/drive/MyDrive/kdt_240424/m6_dl/data/model/best_model.pth')
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('조기 종료 발동.')
            break

# 5단계: 최고의 모델을 로드하고 테스트 데이터셋으로 평가
# torch.load 함수는 지정된 경로에서 모델 파라미터를 불러오며, load_state_dict 메서드를 사용하여 이 파라미터를 현재 net 모델에 로드
net.load_state_dict(torch.load('/content/drive/MyDrive/kdt_240424/m6_dl/data/model/best_model.pth'))
correct = 0 # 정확히 분류된 샘플의 수를 세기 위한 변수
total = 0 # 테스트셋의 전체 샘플 수를 세기 위한 변수
with torch.no_grad():
    for data in testloader: # 테스트 데이터셋을 배치 단위로 순회
        images, labels = data
        outputs = net(images) # 현재 배치의 이미지를 모델에 전달하여 예측값을 계산
        _, predicted = torch.max(outputs.data, 1) # torch.max는 각 예측에 대한 최대값과 그 위치(인덱스)를 반환. 위치만 필요하므로 _를 사용하여 최대값은 무시
        total += labels.size(0) # labels.size(0)는 현재 배치의 크기(샘플 수)
        correct += (predicted == labels).sum().item() # 일치하는 경우의 수를 텐서 형태로 반환하며, .item()으로 이를 파이썬의 스칼라 값으로 변환

print(f'10000개의 테스트 이미지에 대한 네트워크의 정확도: {100 * correct / total} %')



Task1_0829. 가상 데이터 생성 (generate_data 함수) 후 모델링 및 평가하세요
- generate_data 함수는 1000개의 랜덤 시퀀스 데이터를 생성합니다.
  - vocab_size: 시퀀스에 사용할 어휘의 크기를 설정합니다. 여기서는 100개의 단어를 사용합니다.
  - data: 각 시퀀스는 seq_length=10으로 설정된 10개의 정수(단어 인덱스)로 구성됩니다.
  - labels: 각 시퀀스에 대해 0 또는 1의 이진 레이블을 무작위로 할당합니다.

- LSTM 기반 분류 모델을 정의하고, 가상 데이터로 학습 및 검증을 수행합니다.
- 조기 종료를 통해 학습 중 성능이 더 이상 개선되지 않을 때 학습을 중단합니다.
최종적으로 테스트 데이터로 최고 성능의 모델을 평가합니다.

#### nn.Embedding(vocag_size,embed_dim)
- 텍스트 데이터(LSTM 모델): 텍스트 데이터는 이산적인 정수로 표현되므로, 이 정수들을 고차원 벡터로 매핑하는 nn.Embedding 계층이 필요합니다. 이 임베딩 계층은 단어 간의 의미적 유사성을 학습하는 데 유용합니다.

- 이미지 데이터(CNN 모델): 이미지 데이터는 이미 공간적 구조를 가진 연속적인 값(픽셀)으로 표현되므로, 임베딩 계층이 필요하지 않습니다. 대신, 합성곱 계층이 이미지의 패턴을 학습하는 데 사용됩니다.

LSTM 모델의 경우 텍스트 데이터의 정수 인덱스를 벡터로 변환하기 위해 nn.Embedding이 필요하지만, CNN 모델에서는 이미지 데이터를 처리하는 데 이미 직접적인 합성곱 연산이 사용되므로 임베딩 계층이 필요하지 않습니다.

nn.BCEWithLogitsLoss는 PyTorch에서 이진 분류(Binary Classification) 작업을 수행할 때 주로 사용하는 손실 함수입니다. 이 함수는 이진 교차 엔트로피 손실(Binary Cross Entropy Loss)와 시그모이드(Sigmoid) 함수를 결합한 형태로 제공됩니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels

data, labels = generate_data()

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels

data, labels = generate_data()

# 텐서 데이터셋 및 데이터 로더 생성
dataset = TensorDataset(data, labels)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - (train_size + val_size)
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# LSTM 모델 클래스 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        lstm_out, (hidden, _) = self.lstm(embedded)
        hidden = hidden[-1,:,:]
        return self.fc(hidden)

model = LSTMModel(vocab_size=100, embed_dim=50, hidden_dim=100, output_dim=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

# 훈련 함수
def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data).squeeze(1)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# 평가 함수
def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data, labels in data_loader:
            outputs = model(data).squeeze(1)
            loss = criterion(outputs, labels.float())
            total_loss += loss.item()
            predictions = torch.round(torch.sigmoid(outputs))
            correct_predictions = (predictions == labels.unsqueeze(1)).float()
            total_accuracy += correct_predictions.sum().item()
    return total_loss / len(data_loader), total_accuracy / len(data_loader.dataset)

# 조기 종료 로직을 포함한 훈련 및 검증 과정
best_val_loss = float('inf') # float('inf')는 파이썬에서 양의 무한대를 나타내는 방식
patience = 3
trials = 0

for epoch in range(20):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        trials += 1
        if trials >= patience:
            print("조기 종료 발생")
            break

# 테스트 데이터로 최고 모델 평가
model.load_state_dict(torch.load('best_model.pth'))
test_loss, test_accuracy = evaluate(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


Epoch 1, Train Loss: 0.6947, Val Loss: 0.6940, Val Accuracy: 15.3600
Epoch 2, Train Loss: 0.6821, Val Loss: 0.6948, Val Accuracy: 15.3067
Epoch 3, Train Loss: 0.6724, Val Loss: 0.6976, Val Accuracy: 15.2400
Epoch 4, Train Loss: 0.6588, Val Loss: 0.7000, Val Accuracy: 15.2933
조기 종료 발생
Test Loss: 0.6880, Test Accuracy: 15.5733


<ipython-input-2-c109df005c34>:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))
